In [1]:
from tensorflow import keras

import time
import sys
from pathlib import Path
import numpy as np
import os

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')

# from lib.Kmeans_lib import *
# from lib.EvalMetrics import *

from lib.simulation_lib import save_plots
from lib.CustomLayer_lib import Custom_Layer
from lib.utils import create_dataset, RunOneEpoch, TrainSettings
from lib.EvalMetrics import ComputeEvalMetrics2

2022-11-05 19:09:32.115862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# Define settings

settings_train = TrainSettings()
settings_train.verbosity = 'NONE'
settings_train.fill_cmtx = False
settings_train.save_output = False
settings_train.save_plots = False
settings_train.save_extralog = False
settings_train.mode = 'Train'

settings_test = TrainSettings()
settings_test.verbosity = 'EOEINFO'
settings_test.fill_cmtx = True
settings_test.save_output = True
settings_test.save_plots = True
settings_test.save_extralog = True
settings_test.mode = 'Test'

In [11]:
import shutil
shutil.rmtree('Results/', ignore_errors = True)

In [15]:
# File structure:
# /Results/dataset_

if True:
    list_datasets = [[3500, 500]]# Format (n_train, n_test)
    list_features = [10, 30, 50, 100]
    list_batches = [50] #[20, 40, 100]# [5, 10, 20, 40, 100]
    iterations = 3

for n_train, n_test in list_datasets:
    for idx in range(1, iterations+1):

        print("############# STARTING DATASET {} ############".format(idx))

        # Create directory and log file
        if settings_test.save_output == True:      
            RESULTS_PATH = 'Results/ds{}_{}_{}/'.format(idx,n_train, n_test)
            os.makedirs(RESULTS_PATH, exist_ok = True)
            settings_test.save_path = RESULTS_PATH
            
            with open(RESULTS_PATH + 'output_log.txt', 'w') as new_file:  # overwrite if already exists
                # File headers
                new_file.write("# Dataset: {}, n_train = {}, n_test = {} \n".format(idx,n_train, n_test))
                new_file.write("# \n")
                new_file.write("# FEATURES\t; BATCH_SZ\t; CST_ERRS\t; CST_ACC\t; MDL_ERRS\t; MDL_ACC\t;\t TIME\t; CST_ERR_ARRAY\t; MDL_ERR_ARRAY\n")
                new_file.close()
            
            with open(RESULTS_PATH + 'clustering_metrics_log.txt', 'w') as new_file:  # overwrite if already exists
                # File headers
                new_file.write("# Dataset: {}, n_train = {}, n_test = {} \n".format(idx,n_train, n_test))
                new_file.write("# \n")
                new_file.write("# FEATURES\t; BATCH_SZ\t; ACCURACY\t; MIC_PREC\t; MIC_RCLL\t;\t MIC_F1\t; MAC_PREC\t; MAC_RCLL\t; MAC_F1\t; WGT_PREC\t; WGT_RCLL\t; WGT_F1\n")
                new_file.close()
                
            with open(RESULTS_PATH + 'model_metrics_log.txt', 'w') as new_file:  # overwrite if already exists
                # File headers
                new_file.write("# Dataset: {}, n_train = {}, n_test = {} \n".format(idx,n_train, n_test))
                new_file.write("# \n")
                new_file.write("# FEATURES\t; BATCH_SZ\t; ACCURACY\t; MIC_PREC\t; MIC_RCLL\t;\t MIC_F1\t; MAC_PREC\t; MAC_RCLL\t; MAC_F1\t; WGT_PREC\t; WGT_RCLL\t; WGT_F1\n")
                new_file.close()

        # Create dataset
        digits_train, label_digits_train, digits_test, label_digits_test = create_dataset(n_train, n_test)

        for n_feat in list_features:

            print("Starting with n_features: ", n_feat)

            #  Set model and features
            MODEL_PATH = 'Models/{}/'.format(n_feat)
            features_saved = np.loadtxt(MODEL_PATH + 'll_features.txt')
            labels_features_saved = np.loadtxt(MODEL_PATH + 'll_labels_features.txt').astype(int)
            keras_model = keras.models.load_model(MODEL_PATH + 'original_mnist_cnn.h5')  # Original model

            for batch_size in list_batches:
                
                print("Starting batch size: ", batch_size)

                # Initialize model
                Model = Custom_Layer(keras_model)
                Model.title = 'CWR'
                Model.filename = 'CWR'
                Model.l_rate = 0.01
                Model.update_batch_size = 10
                Model.ll_method = 'CWR'
                Model.clustering_batch_size = batch_size

                startTime = time.time()
                
                # Train the model
                Model.settings = settings_train
                RunOneEpoch(Model, digits_train, label_digits_train,features_saved, labels_features_saved)

                # Test the model
                Model.settings = settings_test
                RunOneEpoch(Model, digits_test, label_digits_test,features_saved, labels_features_saved)
                
                endTime = time.time()

                # Save log
                # Unpack datalog output
                clust_err = Model.settings.datalog[0]
                model_err = Model.settings.datalog[1]
                clust_err_array = Model.settings.datalog[2]
                model_err_array = Model.settings.datalog[3]

                if Model.settings.save_output == True:
                    with open(RESULTS_PATH + 'output_log.txt', 'a') as new_file:  
                        new_file.write("\t{:3d} \t;\t {:3d} \t;\t".format(n_feat,batch_size))
                        new_file.write("{:3d} \t;\t {:.1%} \t;\t{:3d} \t;\t {:.1%} \t;\t".format(clust_err, 1-clust_err/n_test, model_err, 1-model_err/n_test))
                        new_file.write("{:.3f}\t;\t".format(endTime-startTime))
                        new_file.write("{}\t;\t{}\n".format(clust_err_array, model_err_array))
                    new_file.close()

                #  Save plots
                if Model.settings.save_plots == True:
                    save_plots(Model, RESULTS_PATH + '/Plots', 'clust','Features_{}_Batch_{}'.format(n_feat, batch_size))
                    save_plots(Model, RESULTS_PATH + '/Plots', 'model','Features_{}_Batch_{}'.format(n_feat, batch_size))

                if Model.settings.save_extralog == True:

                    # Extract output
                    true_labels = Model.settings.extralog[0]
                    pseudo_labels = Model.settings.extralog[1]
                    pred_labels = Model.settings.extralog[2]

                    # Compute Clustering metrics
                    metrics = ComputeEvalMetrics2(true_labels, pseudo_labels)
                    with open(RESULTS_PATH + 'clustering_metrics_log.txt', 'a') as new_file: 
                        new_file.write("\t{:3d} \t;\t {:3d}".format(n_feat,batch_size))
                        for i in range(0, len(metrics)):
                            new_file.write(" \t;\t  {:.2f}".format(metrics[i]))
                        new_file.write("\n")
                        new_file.close()

                    # Compute Model metrics
                    metrics = ComputeEvalMetrics2(true_labels, pred_labels)
                    with open(RESULTS_PATH + 'model_metrics_log.txt', 'a') as new_file: 
                        new_file.write("\t{:3d} \t;\t {:3d}".format(n_feat,batch_size))
                        for i in range(0, len(metrics)):
                            new_file.write(" \t;\t  {:.2f}".format(metrics[i]))
                        new_file.write("\n")
                        new_file.close()


                    #save_plots(Model, RESULTS_PATH + '/Metrics
                    
                    
                    # ', 'clust','Features_{}_Batch_{}'.format(n_feat, batch_size))

                    


                


############# STARTING DATASET 1 ############
Starting with n_features:  10
Starting batch size:  50
Starting Train batch: 1/7
New digit detected -> 6
New digit detected -> 8
New digit detected -> 7
Starting Train batch: 2/7
New digit detected -> 9
Starting Train batch: 3/7
Starting Train batch: 4/7
Starting Train batch: 5/7
Starting Train batch: 6/7
Starting Train batch: 7/7
Starting Test batch: 1/1
Total clustering error: 10.0% (5/50 errors, 90.0% accuracy)
Total model error: 8.0% (4/50 errors, 92.0% accuracy)
Starting with n_features:  30
Starting batch size:  50
Starting Train batch: 1/7
New digit detected -> 6
New digit detected -> 7
New digit detected -> 8
Starting Train batch: 2/7
New digit detected -> 9
Starting Train batch: 3/7
Starting Train batch: 4/7
Starting Train batch: 5/7
Starting Train batch: 6/7
Starting Train batch: 7/7
Starting Test batch: 1/1
Total clustering error: 14.0% (7/50 errors, 86.0% accuracy)
Total model error: 4.0% (2/50 errors, 96.0% accuracy)
Starting w

/Users/andrea/Documents/VS Code/MachineLearning/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/VS Code/MachineLearning/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Starting with n_features:  50
Starting batch size:  50
Starting Train batch: 1/7
New digit detected -> 6
New digit detected -> 8
New digit detected -> 7
New digit detected -> 9
Starting Train batch: 2/7
Starting Train batch: 3/7
Starting Train batch: 4/7
Starting Train batch: 5/7
Starting Train batch: 6/7
Starting Train batch: 7/7
Starting Test batch: 1/1
Total clustering error: 10.0% (5/50 errors, 90.0% accuracy)
Total model error: 6.0% (3/50 errors, 94.0% accuracy)


/Users/andrea/Documents/VS Code/MachineLearning/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/VS Code/MachineLearning/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Starting with n_features:  100
Starting batch size:  50
Starting Train batch: 1/7
New digit detected -> 6
New digit detected -> 7
New digit detected -> 8
New digit detected -> 9
Starting Train batch: 2/7
Starting Train batch: 3/7
Starting Train batch: 4/7
Starting Train batch: 5/7
Starting Train batch: 6/7
Starting Train batch: 7/7
Starting Test batch: 1/1
Total clustering error: 4.0% (2/50 errors, 96.0% accuracy)
Total model error: 2.0% (1/50 errors, 98.0% accuracy)
############# STARTING DATASET 3 ############
Starting with n_features:  10
Starting batch size:  50
Starting Train batch: 1/7
New digit detected -> 7
New digit detected -> 8
New digit detected -> 9
New digit detected -> 6
Starting Train batch: 2/7
Starting Train batch: 3/7
Starting Train batch: 4/7
Starting Train batch: 5/7
Starting Train batch: 6/7
Starting Train batch: 7/7
Starting Test batch: 1/1
Total clustering error: 12.0% (6/50 errors, 88.0% accuracy)
Total model error: 12.0% (6/50 errors, 88.0% accuracy)
Starting 